In [ ]:
%pip install boto3 amazon_transcribe

In [ ]:
import asyncio
import nest_asyncio
import sounddevice
import boto3
import os

from amazon_transcribe.client import TranscribeStreamingClient
from amazon_transcribe.handlers import TranscriptResultStreamHandler
from amazon_transcribe.model import TranscriptEvent

nest_asyncio.apply()

In [ ]:
class MyEventHandler(TranscriptResultStreamHandler):
    async def handle_transcript_event(self, transcript_event: TranscriptEvent):
        # This handler  be implemented to handle transcriptions as needed.
        # Here's an excanample to get started.
        results = transcript_event.transcript.results
        for result in results:
            for alt in result.alternatives:
                print(alt.transcript)

In [ ]:
def create_transcribe_client():
    """Creates a TranscribeStreamingClient, using IAM role credentials if available,
    otherwise using environment variables."""
    try:
        session = boto3.Session()
        client = session.client('transcribe')  # Test for IAM role availability
        return TranscribeStreamingClient(session=session)

    except Exception as e:
        print("IAM role credentials not found, using environment variables.")
        try:
            session = boto3.Session(
                aws_access_key_id=os.environ['AWS_ACCESS_KEY_ID'],
                aws_secret_access_key=os.environ['AWS_SECRET_ACCESS_KEY'],
                region_name="us-east-1"
            )
            return TranscribeStreamingClient(session=session)
        except KeyError:
            print("Environment variables not set for AWS credentials.")
            raise e

In [ ]:
async def mic_stream():
    # This function wraps the raw input stream from the microphone forwarding
    # the blocks to an asyncio.Queue.
    loop = asyncio.get_event_loop()
    input_queue = asyncio.Queue()

    def callback(indata, frame_count, time_info, status):
        loop.call_soon_threadsafe(input_queue.put_nowait, (bytes(indata), status))

    # Be sure to use the correct parameters for the audio stream that matches
    # the audio formats described for the source language you'll be using:
    # https://docs.aws.amazon.com/transcribe/latest/dg/streaming.html
    stream = sounddevice.RawInputStream(
        channels=1,
        samplerate=16000,
        callback=callback,
        blocksize=1024 * 2,
        dtype="int16",
    )
    # Initiate the audio stream and asynchronously yield the audio chunks
    # as they become available.
    with stream:
        while True:
            indata, status = await input_queue.get()
            yield indata, status

In [ ]:
async def write_chunks(stream):
    # This connects the raw audio chunks generator coming from the microphone
    # and passes them along to the transcription stream.
    async for chunk, status in mic_stream():
        await stream.input_stream.send_audio_event(audio_chunk=chunk)
    await stream.input_stream.end_stream()

In [ ]:
async def basic_transcribe():
    # Setup up our client with our chosen AWS region
    client = create_transcribe_client()

    # Start transcription to generate our async stream
    stream = await client.start_stream_transcription(
        language_code="en-US",
        media_sample_rate_hz=16000,
        media_encoding="pcm"
    )

    # Instantiate our handler and start processing events
    handler = MyEventHandler(stream.output_stream)
    await asyncio.gather(write_chunks(stream), handler.handle_events())

In [ ]:
asyncio.run(basic_transcribe())